In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.circuit import Gate
from qiskit import schedule as build_schedule
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_manila')

In [3]:
#IBMProvider.save_account(token="dd5975c35d22e23d5a6666f97ad3f54143b2de99b9a3cbbf7588a8d7cf508efaa9c140872a0af59ad5e334585fa7519044162b94c91b70d6ec07c777d67ff92b")

In [4]:
# Loading your IBM Q account(s)
#provider = IBMProvider()
#provider = IBMProvider(instance="ibm-q/open/main")
#backend = provider.get_backend('ibmq_manila')

In [5]:
backend_config = backend.configuration()
backend_defaults = backend.defaults()

In [6]:
qind = 0
cmap = []
for i, j in backend_config.coupling_map:
    if i == qind:
        cmap.append([i, j])
pair = cmap[0]
con_chan = backend_config.control(pair)[0]

In [7]:
from scipy.optimize import leastsq,minimize, curve_fit

# samples need to be multiples of 16 to accommodate the hardware limitations
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

# lorentzian function
def lorentzian(f, f0, k, a, offs):
    return a*k/(2*np.pi)/((k/2)**2+(f-f0)**2)+offs

#fit_lorentzian takes two arrays that contain the frequencies and experimental output values of each frequency respectively. 
#returns the lorentzian parameters that best fits this output of the experiment.
#popt are the fit parameters and pcov is the covariance matrix for the fit
def fit_lorentzian(freqs,values):
    p0=[freqs[np.argmax(values)],(freqs[-1]-freqs[0])/2,max(values),0]
    bounds=([freqs[0],0,0,-np.inf],[freqs[-1],freqs[-1]-freqs[0],np.inf,np.inf])
    popt,pcov=curve_fit(lorentzian, freqs, values, p0=p0, bounds=bounds)
    return popt,pcov

# Gaussian function
def gaussian(f, f0, sigma, a, offs):
    return a*np.exp(-(f-f0)**2/(2*sigma**2))+offs

#fit_gaussian takes two arrays that contain the frequencies and experimental output values of each frequency respectively. 
#returns the gaussian parameters that best fits this output of the experiment.
#popt are the fit parameters and pcov is the covariance matrix for the fit
def fit_gaussian(freqs,values):
    p0=[freqs[np.argmax(values)],(freqs[-1]-freqs[0])/2,max(values),0]
    bounds=([freqs[0],0,0,-np.inf],[freqs[-1],freqs[-1]-freqs[0],np.inf,np.inf])
    popt,pcov=curve_fit(gaussian, freqs, values, p0=p0, bounds=bounds)
    return popt,pcov

# normalize the data points to fall in the range of [0,1]
def normalize(a):
    a= a-min(a)
    return a/max(a)

In [8]:
from qiskit import pulse            # This is where we access all of our Pulse features!
from qiskit.circuit import Parameter
import numpy as np

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc.)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

qubit=0   # qubit used in our experiment

qubit_drive_sigma = 0.1 * us        #the width of the qubit spectroscopy drive
stark_tone_drive_sigma=10 * ns      #This is Gaussian sigma of rising and falling edge
drive_duration=8*qubit_drive_sigma  #the stark drive duration

start=4.960 * GHz  # qubit spectroscopy start frequency
stop=4.980 * GHz   # qubit spectroscopy stop frequency
freqs = np.linspace(start, stop, 41)

In [9]:
# pulse sequence for the experiment at different amplitudes
amplitude = Parameter('amplitude')
drive_freq = Parameter('drive_freq')
control_freq = Parameter('control_freq')
with pulse.build(backend=backend, name='ac Stark Shift Experimet') as stark_pulse:
    duration = get_closest_multiple_of_16(pulse.seconds_to_samples(drive_duration))
    drive_qubit_sigma = pulse.seconds_to_samples(qubit_drive_sigma)
    drive_stark_tone_sigma = pulse.seconds_to_samples(stark_tone_drive_sigma)
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(drive_freq, drive_chan)
    pulse.play(pulse.Gaussian(duration=duration,
                              amp = 0.05,
                              sigma=drive_qubit_sigma,
                              name='qubit tone'), drive_chan)
    pulse.set_frequency(control_freq, con_chan)
    pulse.play(pulse.GaussianSquare(duration = duration,
                                    amp = amplitude,
                                    sigma = drive_stark_tone_sigma,
                                    risefall_sigma_ratio = 2,
                                    name = 'stark tone'), con_chan)

In [10]:
stark_spect_gate = Gate("stark", 1, [amplitude, drive_freq, control_freq])

qc_stark = QuantumCircuit(1, 1)

qc_stark.append(stark_spect_gate, [0])
qc_stark.measure(0, 0)
qc_stark.add_calibration(stark_spect_gate, (0,), stark_pulse)

In [14]:
stark_tone_amplitude = np.linspace(0, 0.2, 11) #change to amplitude

In [16]:
from qiskit.tools.monitor import job_monitor

num_shots = 4*1024

stark_tone_results=[]
amp = 0.02
#for amp in stark_tone_amplitude[1]:
qc_stark_circs = [qc_stark.assign_parameters({amplitude: amp , drive_freq: freq, control_freq: freq - 100 * MHz}, inplace=False) for freq in freqs]
job = backend.run(qc_stark_circs, 
                  meas_level=1, 
                  meas_return='avg', 
                  shots=num_shots)
job_monitor(job)
job.error_message()    

Job Status: job incurred error     


'Internal Error. Error code: 9999.'